<a href="https://colab.research.google.com/github/Maruf-16203091/weather-app/blob/main/weather_detection_vlm_p_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
from tqdm import tqdm
from google.colab import drive, files

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

drive.mount('/content/drive')
train_dir = "/content/drive/MyDrive/weather_data/train"
val_dir   = "/content/drive/MyDrive/weather_data/val"


train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2,0.2,0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes
num_classes = len(class_names)

print("Weather classes:", class_names)



Using device: cuda
Mounted at /content/drive
Weather classes: ['clear', 'foggy', 'overcast', 'partly cloudy', 'rainy', 'snowy', 'unknown']


In [2]:
# ===============================
# 7. MODEL: RESNET-18
# ===============================
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# ===============================
# 8. LOSS, OPTIMIZER, SCHEDULER
# ===============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# ===============================
# 9. TRAIN ONE EPOCH
# ===============================
def train_one_epoch(model, loader):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for imgs, labels in tqdm(loader, desc="Training", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return total_loss / len(loader), 100 * correct / total

# ===============================
# 10. VALIDATION
# ===============================
def evaluate(model, loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Validation", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return total_loss / len(loader), 100 * correct / total

# ===============================
# 11. TRAINING LOOP
# ===============================
num_epochs = 15

train_losses, val_losses = [], []
train_accs, val_accs = [], []

best_val_acc = 0.0
best_model_path = "/content/drive/MyDrive/weather_data/weather_resnet18_best.pth"

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = evaluate(model, val_loader)
    scheduler.step()

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    print(
        f"Epoch [{epoch+1}/{num_epochs}] | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%"
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Best model saved (Val Acc: {best_val_acc:.2f}%)")

# ===============================
# 12. SAVE FINAL MODEL
# ===============================
final_model_path = "/content/drive/MyDrive/weather_data/weather_resnet18_final.pth"
torch.save(model.state_dict(), final_model_path)
print("✅ Final model saved at:", final_model_path)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 199MB/s]


Epoch [1/15] | Train Loss: 1.1314, Train Acc: 57.19% | Val Loss: 0.9030, Val Acc: 66.95%
✅ Best model saved (Val Acc: 66.95%)


Epoch [2/15] | Train Loss: 0.7736, Train Acc: 71.32% | Val Loss: 0.7623, Val Acc: 71.62%
✅ Best model saved (Val Acc: 71.62%)


...

Epoch [11/15] | Train Loss: 0.1194, Train Acc: 96.99% | Val Loss: 0.8560, Val Acc: 74.10%


Epoch [12/15] | Train Loss: 0.1019, Train Acc: 97.68% | Val Loss: 0.8465, Val Acc: 74.29%
✅ Best model saved (Val Acc: 74.29%)


Epoch [13/15] | Train Loss: 0.1028, Train Acc: 97.41% | Val Loss: 0.8701, Val Acc: 74.29%


Epoch [14/15] | Train Loss: 0.0885, Train Acc: 97.92% | Val Loss: 0.8819, Val Acc: 73.90%


Epoch [15/15] | Train Loss: 0.0819, Train Acc: 98.33% | Val Loss: 0.8823, Val Acc: 74.19%
✅ Final model saved at: /content/drive/MyDrive/weather_data/weather_resnet18_final.pth


In [3]:
LLM_NAME = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(LLM_NAME).to(device)

print("✅ FLAN-T5 Large Loaded")
WEATHER_HINTS = {
    "clear": "clear sky with bright sunlight",
    "foggy": "low visibility due to fog",
    "overcast": "cloud-covered sky",
    "partly_cloudy": "mix of sun and clouds",
    "rainy": "rainfall and wet roads",
    "snowy": "snow-covered environment"
}
WEATHER_ADVICE = {
    "clear": "Normal driving conditions. Maintain standard speed limits.",
    "foggy": "Reduce speed to 40–50 km/h. Use fog lights and keep safe distance.",
    "overcast": "Drive cautiously. Roads may become slippery.",
    "partly_cloudy": "Normal conditions, but stay alert to changes.",
    "rainy": "Reduce speed to 50 km/h. Avoid sudden braking.",
    "snowy": "Limit speed to 30 km/h. Use snow tires if possible."
}
def generate_weather_reasoning(weather):
    prompt = f"""
You are a weather and road safety expert.

Detected weather condition: {weather}
Visual cues: {WEATHER_HINTS.get(weather)}

Write a short paragraph (3–4 sentences) explaining:
- Why this weather occurs
- How it affects visibility and road safety
- General precautions for people

Write only weather-specific explanation.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = t5_model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.3,
        top_p=0.9,
        do_sample=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ FLAN-T5 Large Loaded


In [4]:
CONFIDENCE_THRESHOLD = 0.4
ENTROPY_THRESHOLD = 2.0
TEMPERATURE = 2.0

def predict_weather(model, image_tensor):
    image_tensor = image_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(image_tensor)
        probs = F.softmax(logits / TEMPERATURE, dim=1)

        confidence, idx = torch.max(probs, dim=1)
        entropy = -(probs * torch.log(probs + 1e-8)).sum()

    if confidence < CONFIDENCE_THRESHOLD or entropy > ENTROPY_THRESHOLD:
        return None, confidence.item()

    return class_names[idx.item()], confidence.item()



In [5]:
def analyze_weather(image_path):
    img = Image.open(image_path).convert("RGB")
    img = val_transforms(img)

    weather, confidence = predict_weather(model, img)

    if weather is None:
        return "⚠️ Low confidence prediction. Please upload a clearer image."

    advice = WEATHER_ADVICE[weather]
    reasoning = generate_weather_reasoning(weather)

    return f"""
🌤 Weather Detected: {weather}
📊 Confidence: {confidence*100:.2f}%

🚗 Advice:
{advice}

🧠 Explanation:
{reasoning}
"""

In [9]:
uploaded = files.upload()

for file in uploaded.keys():
    print("="*60)
    print(analyze_weather(file))
    print("="*60)

Saving 11b47a8a-0dffe086.jpg to 11b47a8a-0dffe086.jpg

🌤 Weather Detected: overcast
📊 Confidence: 44.08%

🚗 Advice:
Drive cautiously. Roads may become slippery.

🧠 Explanation:
Overcast weather is a type of weather that occurs when the sky is cloudy and the sun is not visible. Overcast weather is a type of weather that occurs when the sun is not visible. Overcast weather is a type of weather that occurs when the sky is cloudy and the sun is not visible.

